# Results 

### Analysis of the final data set  
After the loading, merging, and pre-processing of our 3 datasets, we have the final `final_merged_data.csv` dataset composed of:

|       | Movie name           | Movie release date | Movie box office revenue | Movie runtime | Movie languages | Movie countries | Movie genres | Movie score | Movie votes | Movie director | Movie star | Movie budget | Movie gross | Movie company | Number of nominations | Nomination winner | Budget/revenues ratio |
|-------|----------------------|--------------------|--------------------------|---------------|-----------------|-----------------|--------------|-------------|-------------|-----------------|------------|--------------|-------------|---------------|----------------------|-------------------|-----------------------|
| Type of values |   Str   | Int                  | Float                 |                        | D             | A               | B               | C            | D           | A           | B               | C          | D            | A           | B             | C                    | D                 |                       |
| Example |       | 1                    | 5                  | 2                        | 0             | P0              | 0               | 0            | 1           | 2           | 0               | 0          | 1            | 2           | 0             | 0                    | 1                 |                       |
| Number of rows with this information |       | 0                    | 0                  | 1                        | 2             | P1              | 1               | 0            | 0           | 1           | 7               | 5          | 0            | 0           | 6             | 5                    | 0                 | 6                     |
| Number of NaN |       | 1                    | 3                  | 5                        | 4             | P2              | 2               | 3            | 5           | 6           | 0               | 6          | 5            | 2           | 0             | 6                    | 5                 |                       |
| Statistical information (mean, min, max) |       | 0                    | 6                  | 3                        | 2             | P3              | 0               | 6            | 5           | 2           | 0               | 6          | 5            | 6           | 0             | 6                    | 6                 |                       |


| Header 1 | Header 2 | Header 3 |
|----------|----------|----------|
| Row 1    | Data     | Data     |
| Row 2    | Data     | Data     |


In [6]:
import pandas as pd 

data = pd.read_csv('data/final_merged_data.csv')
data.dtypes

Movie name                   object
Movie release date          float64
Movie box office revenue    float64
Movie runtime               float64
Movie languages              object
Movie countries              object
Movie genres                 object
Movie score                 float64
Movie votes                 float64
Movie director               object
Movie star                   object
Movie budget                float64
Movie gross                 float64
Movie company                object
Number of nomination        float64
Nomination winner            object
Budget/revenues ratio       float64
dtype: object